# Как оценивать квантили в АБ-тестах (метод [Spotify](https://engineering.atspotify.com/2022/03/comparing-quantiles-at-scale-in-online-a-b-testing/))

Предполагается, что при стандартном бутстрепе сложность будет вида "сгенерировать выборку", "посчитать квантиль по выборке". Это как будто бы много. Вместо этого Spotify предлагает отсортировать выборку 1 раз и использовать ее в расчетах. 

<img src = "img/Bootstrapped-Distributon-of-Quantile-Indexes-1536x960.png" width = "800">

In [2]:
import numpy as np
from scipy.stats import binom

alpha=.05
quantile_of_interest=0.5
sample_size=10000
number_of_bootstrap_samples=1000
outcome_sorted = np.sort(np.random.normal(1,1,sample_size))

ci_indexes = binom.ppf([alpha/2,1-alpha/2],sample_size+1, quantile_of_interest)
bootstrap_confidence_interval = outcome_sorted[[int(np.floor(ci_indexes[0])), int(np.ceil(ci_indexes[1]))]]

f"The sample median is {np.quantile(outcome_sorted, quantile_of_interest)}, the {(1-alpha)*100}%\
confidence interval is given by ({bootstrap_confidence_interval})."

'The sample median is 1.00261616216872, the 95.0%confidence interval is given by ([0.97503455 1.02812291]).'

In [6]:
import numpy as np
from numpy.random import normal, binomial

alpha=.05
quantile_of_interest=0.5
sample_size=10000
number_of_bootstrap_samples=10000
outcome_control_sorted = np.sort(normal(1,1,sample_size))
outcome_treatment_sorted = np.sort(normal(1.2,1,sample_size))

bootstrap_difference_distribution = outcome_treatment_sorted[binomial(sample_size+1, 
                                                                      quantile_of_interest, 
                                                                      number_of_bootstrap_samples)] 
- outcome_control_sorted[binomial(sample_size+1,
                                  quantile_of_interest, 
                                  number_of_bootstrap_samples)]

bootstrap_confidence_interval = np.quantile(bootstrap_difference_distribution,
[alpha/2 , 1-alpha/2])

f"The sample difference-in-medians is \
{np.quantile(outcome_treatment_sorted, quantile_of_interest)-np.quantile(outcome_control_sorted, quantile_of_interest)},\
the {(1-alpha)*100}% confidence interval for the difference-in-medians is given by ({bootstrap_confidence_interval})."

'The sample difference-in-medians is 0.21418733857943772,the 95.0% confidence interval for the difference-in-medians is given by ([1.18768264 1.23540676]).'

In [5]:
alpha=.05
quantile_of_interest=0.5
sample_size=10000
number_of_bootstrap_samples=1000000
replications = 1000

bootstrap_confidence_intervals = []
ci_index = int(np.floor(binom.ppf(alpha,sample_size+1, quantile_of_interest)))
for i in range(replications):
    outcome_sorted = np.sort(np.random.normal(0,1,sample_size))
    bootstrap_confidence_intervals.append(outcome_sorted[ci_index])

f"The empirical false positive rate of the test using the bootstrap confidence interval is {np.mean([1 if i>0  else 0 for i in bootstrap_confidence_intervals])*100}%, the intended false positive rate is {alpha*100}%"

'The empirical false positive rate of the test using the bootstrap confidence interval is 5.16%, the intended false positive rate is 5.0%'

In [51]:
import scipy.stats as sps
from tqdm.notebook import tqdm # tqdm – библиотека для визуализации прогресса в цикле
from statsmodels.stats.proportion import proportion_confint
import numpy as np

 
mann_bad_cnt = 0
ttest_bad_cnt = 0
sz = 10000
 
for i in tqdm(range(sz)):
    test_zero_array = sps.bernoulli(p=0.55).rvs(1000)
    control_zero_array = sps.bernoulli(p=0.5).rvs(1000)
    test = sps.expon(loc=0, scale=6).rvs(1000) * test_zero_array # ET = 3.3
    control = sps.expon(loc=0, scale=7).rvs(1000) * control_zero_array # EC = 3.5
     
    # Проверяем гипотезу
    mann_pvalue = sps.mannwhitneyu(control, test, alternative='less').pvalue
    ttest_pvalue = sps.ttest_ind(control, test, alternative='less').pvalue
    if mann_pvalue < 0.05:
        mann_bad_cnt += 1
 
    if ttest_pvalue < 0.05:
        ttest_bad_cnt += 1


left_mann_power, right_mann_power = proportion_confint(count = mann_bad_cnt, nobs = sz, alpha=0.05, method='wilson')
left_ttest_power, right_ttest_power = proportion_confint(count = ttest_bad_cnt, nobs = sz, alpha=0.05, method='wilson')
# Выводим результаты
print(f"Mann-whitneyu LESS power: {round(mann_bad_cnt / sz, 4)}, [{round(left_mann_power, 4)}, {round(right_mann_power, 4)}]")
print(f"T-test LESS power: {round(ttest_bad_cnt / sz, 4)}, [{round(left_ttest_power, 4)}, {round(right_ttest_power, 4)}]")

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
def mannwhitney(a, b):
    """
    Двусторонний тест Манна-Уитни
    :param a: np.array вида (n_experiments, n_users), значения метрики в контроле
    :param b: np.array вида (n_experiments, n_users), значен я метрики в тесте
    :return: np.array вида (n_experiments), двусторонние p-value методом Манна-Уитни для всех экспериментов
    """
    result = list(map(lambda x: scipy.stats.mannwhitneyu(
        x[0], x[1], alternative='two-sided').pvalue, zip(a, b)))
    return np.array(result)

